In [1]:
import math  
import pandas as pd  
import numpy as np  
import requests  
import json
import sklearn
import nltk
import re

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

from sklearn import tree
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_selection import chi2

from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline

from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest

from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support

/Users/lheinle/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Let's grab the tweets

In [7]:
import os
import twitter

ImportError: No module named 'twitter'

In [9]:
twitter_key = os.environ.get('TWITTER_CONSUMER_KEY')
twitter_secret = os.environ.get('TWITTER_CONSUMER_SECRET')
twitter_token = os.environ.get('TWITTER_ACCESS_TOKEN')
twitter_token_secret = os.environ.get('TWITTER_ACCESS_TOKEN_SECRET')

In [11]:
api = twitter.Api(consumer_key=twitter_key,
                  consumer_secret=twitter_secret,
                  access_token_key=twitter_token,
                  access_token_secret=twitter_token_secret)

In [16]:
print(api.VerifyCredentials())

{"created_at": "Sat Sep 15 22:12:06 +0000 2012", "description": "Data viz @TheBuffaloNews | @UNL_CoJMC grad with interests in coffee, coding & data journalism | Former @OWHNews fellow, #WebDesign intern @Nelnet", "favourites_count": 1420, "followers_count": 509, "friends_count": 1190, "geo_enabled": true, "id": 826036134, "lang": "en", "listed_count": 29, "location": "Buffalo, NY", "name": "Lexie Heinle \ud83d\udc69\ud83c\udffd\u200d\ud83d\udcbb", "profile_background_color": "313278", "profile_background_image_url": "http://pbs.twimg.com/profile_background_images/570057266628280320/z1YQD5bg.jpeg", "profile_banner_url": "https://pbs.twimg.com/profile_banners/826036134/1465071239", "profile_image_url": "http://pbs.twimg.com/profile_images/659729207317041152/BPMXIPph_normal.jpg", "profile_link_color": "6DBDB3", "profile_sidebar_fill_color": "FA0A6A", "profile_text_color": "FA7550", "screen_name": "lexieheinle", "status": {"created_at": "Wed Jun 28 21:27:33 +0000 2017", "id": 8801758618734

In [21]:
print(len())


131


In [25]:
tweets=api.GetUserTimeline('826036134', include_rts=False, count=200, trim_user=True)
print(tweets[0])
print(len(tweets))

{"created_at": "Sat Jun 17 00:14:09 +0000 2017", "hashtags": [{"text": "attjukebox"}], "id": 875869134302720000, "id_str": "875869134302720000", "in_reply_to_screen_name": "buffalobison", "in_reply_to_user_id": 62187116, "lang": "es", "place": {"attributes": {}, "bounding_box": {"coordinates": [[[-78.912276, 42.826008], [-78.79485, 42.826008], [-78.79485, 42.966451], [-78.912276, 42.966451]]], "type": "Polygon"}, "contained_within": [], "country": "United States", "country_code": "US", "full_name": "Buffalo, NY", "id": "a307591cd0413588", "name": "Buffalo", "place_type": "city", "url": "https://api.twitter.com/1.1/geo/id/a307591cd0413588.json"}, "source": "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>", "text": "@buffalobison Despacito #attjukebox", "urls": [], "user": {"id": 826036134}, "user_mentions": [{"id": 62187116, "name": "Jon Turner", "screen_name": "buffalobison"}]}
131


In [45]:
cleaner_tweets = []
for tweet in tweets:
    cleaner_tweets.append({'id': tweet.id, 'text': tweet.text, 'created_at': tweet.created_at})
    
df = pd.DataFrame(cleaner_tweets)

In [53]:
df.head()

,created_at,id,text
0,Sat Jun 17 00:14:09 +0000 2017,875869134302720000,@buffalobison Despacito #attjukebox
1,Thu Jun 15 23:47:05 +0000 2017,875499936590303232,@tackjhompson I may be absurdly proud of my ti...
2,Thu Jun 15 23:01:56 +0000 2017,875488575726530561,Colorful crowd at the #CorporateChallenge http...
3,Thu Jun 15 14:26:34 +0000 2017,875358879512748032,Will I nerd out about one of my favorite proje...
4,Mon Jun 12 16:05:02 +0000 2017,874296493729099778,Ditto... https://t.co/VMuYoS0ujL


In [54]:
from datetime import datetime
df['created_at'] = df['created_at'].apply(lambda x: datetime.strptime(x , "%a %b %d %H:%M:%S %z %Y"))

In [55]:
df.head()

,created_at,id,text
0,2017-06-17 00:14:09+00:00,875869134302720000,@buffalobison Despacito #attjukebox
1,2017-06-15 23:47:05+00:00,875499936590303232,@tackjhompson I may be absurdly proud of my ti...
2,2017-06-15 23:01:56+00:00,875488575726530561,Colorful crowd at the #CorporateChallenge http...
3,2017-06-15 14:26:34+00:00,875358879512748032,Will I nerd out about one of my favorite proje...
4,2017-06-12 16:05:02+00:00,874296493729099778,Ditto... https://t.co/VMuYoS0ujL


In [2]:
def preProcessing(titles):
    num_titles = titles.size
    clean_titles = []
    stops = set(stopwords.words("english"))
    for i in xrange( 0, num_titles):
        letters_only = re.sub("[^a-zA-Z]", " ", titles[i]) 
        words = titles[i].lower().split()
        words = [w.lower() for w in words if not w in stops]  
        clean_titles.append(" ".join(words))
    return clean_titles

In [3]:
df['clean_text'] = df['text'].apply(lambda x: preProcessing(x))

NameError: name 'df' is not defined